## 数字识别

* 有监督；
* 分类问题；
* 训练数据集：对应数字以及每个位置的像素点组成；
* 测试数据集：每个位置的像素点；
* 提交格式：ImageId,Label；

## 初始化环境

In [1]:
import os,sys,time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False

%matplotlib inline

## 加载、展示数据

In [21]:
def concat_df(train_data, test_data):
    # Returns a concatenated df of training and test set on axis 0
    return pd.concat([train_data, test_data], sort=True).reset_index(drop=True)

def divide_df(all_data, train_size=42000):
    # Returns divided dfs of training and test set
    return all_data.loc[:train_size-1], all_data.loc[train_size:].drop(['label'], axis=1)

In [22]:
train_data = pd.read_csv('input/train.csv')
print train_data.info()

test_data = pd.read_csv('input/test.csv')
print test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28000 entries, 0 to 27999
Columns: 784 entries, pixel0 to pixel783
dtypes: int64(784)
memory usage: 167.5 MB
None


In [23]:
all_data = concat_df(train_data, test_data)

In [24]:
train_data[['label','pixel0']].groupby('label').count()

,pixel0
label,
0,4132
1,4684
2,4177
3,4351
4,4072
5,3795
6,4137
7,4401
8,4063


看到各个数字对应的训练数据个数相差不大，基本都在4000左右；

## 可视化特征

观察任意两组数据的标准差方差比较；

In [25]:
train_data.groupby('label').std().mean(axis=1)

label
0    44.810023
1    24.077799
2    47.097188
3    43.168668
4    40.773742
5    44.821169
6    40.953832
7    38.045724
8    42.534838
9    37.799291
dtype: float64

In [26]:
train_data[train_data['label']%4==0].std().mean()

48.453327804548906

看到std有明显增高，说明各个分类之间的差异是明显的；

## 数据预处理

因为数据无缺失，且为像素数据，不好直接处理，先略过，后续考虑是否有好的方式可以处理；

## 特征工程TODO

同样的，因为是像素数据，不好直接直接特征构建、选择等，不过可以将全是0的特征去除，这些特征明显无法起到帮助作用（即方差过滤法的简单形式）；

0. 方差过滤法；
1. 每组数据都是一个N\*M的分辨率图片的每个像素点，那么可以找到每一行1的个数，每一列1的个数；
2. 1的总数；

In [27]:
all_zero_columns = []
for col in all_data.columns:
    if 0 >= all_data[col].max():
        all_zero_columns.append(col)
print all_zero_columns

['pixel0', 'pixel1', 'pixel10', 'pixel11', 'pixel111', 'pixel112', 'pixel140', 'pixel16', 'pixel168', 'pixel17', 'pixel18', 'pixel19', 'pixel2', 'pixel20', 'pixel21', 'pixel22', 'pixel23', 'pixel24', 'pixel25', 'pixel26', 'pixel27', 'pixel28', 'pixel29', 'pixel3', 'pixel30', 'pixel31', 'pixel4', 'pixel476', 'pixel5', 'pixel52', 'pixel53', 'pixel54', 'pixel55', 'pixel56', 'pixel560', 'pixel57', 'pixel6', 'pixel644', 'pixel671', 'pixel672', 'pixel673', 'pixel699', 'pixel7', 'pixel700', 'pixel701', 'pixel727', 'pixel728', 'pixel729', 'pixel730', 'pixel754', 'pixel755', 'pixel756', 'pixel757', 'pixel758', 'pixel759', 'pixel780', 'pixel781', 'pixel782', 'pixel783', 'pixel8', 'pixel82', 'pixel83', 'pixel84', 'pixel85', 'pixel9']


In [28]:
all_data.drop(all_zero_columns, axis=1, inplace=True)
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Columns: 720 entries, label to pixel99
dtypes: float64(1), int64(719)
memory usage: 384.5 MB


看到特征减少了；

## 划分数据集

In [30]:
train_data, test_data = divide_df(all_data)

In [31]:
from sklearn.model_selection import train_test_split

x,y = train_data.drop(['label'], axis=1),train_data['label']
x_train,x_valid,y_train,y_valid = train_test_split(x,y,test_size=0.3,random_state=0)

x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29400 entries, 26437 to 2732
Columns: 719 entries, pixel100 to pixel99
dtypes: int64(719)
memory usage: 161.5 MB


## 模型构建、优化

### kNN

此问题虽然数据量不算小但是也不大，最简单的方法就是利用kNN，且kNN的一个好处是目前所有特征我们可以假设其权重一致，且分类平均，个类别差异较大，可以看到效果已经很好了，但是我们也知道一方面提升很困难，对于kNN除了修改k、距离公式（将特征权重改变）没有太多其他办法，同时算法运行时间比较长也是kNN的弊端，后面需要优化只能通过换其他算法；

In [32]:
from sklearn.neighbors import KNeighborsClassifier as kNN

knn = kNN()
knn.fit(x_train, y_train)
score = knn.score(x_valid, y_valid)
print 'Accuracy:'+str(score)

Accuracy:0.9657142857142857


### 多模型TODO

### 模型融合TODO

## 结果生成、提交

In [33]:
result = knn.predict(test_data)
result[:10]

array([2., 0., 9., 9., 3., 7., 0., 3., 0., 3.])

In [37]:
[idx-41999 for idx in list(test_data.index)]

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [39]:
pd.DataFrame({'ImageId':[idx-41999 for idx in list(test_data.index)], 'Label':[int(label) for label in list(result)]}).to_csv('output/submission-digit-0917-V-1.0.csv', index=False)